In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d nelgiriyewithana/emotions

 77% 12.0M/15.7M [00:00<00:00, 21.6MB/s]
100% 15.7M/15.7M [00:00<00:00, 16.8MB/s]


In [ ]:
from zipfile import ZipFile
dataset = '/content/emotions.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('Extracted')

Extracted


In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
table=pd.read_csv('text.csv')

In [ ]:
table.head()

,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4


In [ ]:
table.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
max([len(i.split()) for i in table['text']])

178

In [ ]:
min([len(i.split()) for i in table['text']])

1

In [ ]:
import nltk
from nltk.stem import PorterStemmer
nltk.download('punkt')
ps=PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def stemmer(text):
  new_text=[]
  for word in text.split():
    word=ps.stem(word)
    new_text.append(word)
  return ' '.join(new_text)

In [ ]:
table['text']=table['text'].apply(stemmer)

In [ ]:
table.head()

,text,label
0,i just feel realli helpless and heavi heart,4
1,ive enjoy be abl to slouch about relax and unw...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import models, layers

In [ ]:
tokenizer=Tokenizer()

In [ ]:
tokenizer.fit_on_texts(table['text'])

In [ ]:
len(tokenizer.word_index)

51933

In [ ]:
sequences=tokenizer.texts_to_sequences(table['text'])

In [ ]:
max([len(i) for i in sequences])

178

In [ ]:
min([len(i) for i in sequences])

1

In [ ]:
padded_sequences=pad_sequences(sequences,maxlen=max([len(i) for i in sequences]),padding='post')

In [ ]:
padded_sequences.shape

(416809, 178)

In [ ]:
padded_sequences

array([[  1,  30,   2, ...,   0,   0,   0],
       [ 78, 266,  14, ...,   0,   0,   0],
       [  1, 856,  43, ...,   0,   0,   0],
       ...,
       [  1,   2,  47, ...,   0,   0,   0],
       [  1,   2,   7, ...,   0,   0,   0],
       [  1,  58,   7, ...,   0,   0,   0]], dtype=int32)

**Six categories: sadness (0), joy (1), love (2), anger (3), fear (4), and surprise (5)**

In [ ]:
table['label']

0         4
1         0
2         4
3         0
4         4
         ..
416804    2
416805    3
416806    5
416807    3
416808    5
Name: label, Length: 416809, dtype: int64

In [ ]:
y=pd.get_dummies(table['label'])

In [ ]:
X=padded_sequences

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=21)

In [ ]:
X_train.shape,X_test.shape

((312606, 178), (104203, 178))

Building LSTM model

In [ ]:
model=models.Sequential()

In [ ]:
model.add(layers.Embedding(len(tokenizer.word_index)+1,100,mask_zero=True))
model.add(layers.LSTM(100,dropout=0.2))
model.add(layers.Dense(100,activation='relu'))
model.add(layers.Dropout(0.4))
model.add(layers.Dense(6,activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,epochs=5,validation_split=0.2)

Epoch 1/5
7816/7816 [==============================] - 116s 14ms/step - loss: 0.2772 - accuracy: 0.8825 - val_loss: 0.1398 - val_accuracy: 0.9266
Epoch 2/5
7816/7816 [==============================] - 92s 12ms/step - loss: 0.1339 - accuracy: 0.9286 - val_loss: 0.1271 - val_accuracy: 0.9299
Epoch 3/5
7816/7816 [==============================] - 96s 12ms/step - loss: 0.1194 - accuracy: 0.9326 - val_loss: 0.1234 - val_accuracy: 0.9300
Epoch 4/5
7816/7816 [==============================] - 93s 12ms/step - loss: 0.1100 - accuracy: 0.9362 - val_loss: 0.1276 - val_accuracy: 0.9295
Epoch 5/5
7816/7816 [==============================] - 94s 12ms/step - loss: 0.1032 - accuracy: 0.9389 - val_loss: 0.1303 - val_accuracy: 0.9291


Evaluating testing data

In [ ]:
loss,accuracy=model.evaluate(X_test,y_test)

3257/3257 [==============================] - 17s 5ms/step - loss: 0.1308 - accuracy: 0.9296


In [ ]:
accuracy

0.9295701384544373

Overall prediction

In [ ]:
pred = model.predict(X)

13026/13026 [==============================] - 45s 3ms/step


In [ ]:
pred = np.array([np.argmax(x) for x in pred])

In [ ]:
pred

array([4, 0, 4, ..., 5, 3, 5])

Overall accuracy of the model

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(table['label'],pred)

0.9378036462744327

In [ ]:
label_name=['sadness','joy','love','anger','fear','surprise']

In [ ]:
import pickle

In [ ]:
pickle.dump(model,open('emotion_model.pkl','wb'))

In [ ]:
import string

In [ ]:
def punct_rem(text):
  for i in string.punctuation:
    text=text.replace(i,'')
  return text

In [ ]:
text="I am feeling very bad right now I don't know what I am doing with my life"
text=punct_rem(text)
text=stemmer(text)
text=tokenizer.texts_to_sequences([text])
prediction=model.predict(text)
print(prediction)
prediction=np.argmax(prediction)
label_name[prediction]

1/1 [==============================] - 0s 89ms/step
[[9.9999928e-01 1.7723202e-08 7.8174933e-10 6.4965957e-07 1.4840133e-07
  1.4856367e-14]]


'sadness'

In [ ]:
text="A radiant smile adorned her face as she danced freely in the warm embrace of the sunlit meadow, feeling an overwhelming sense of happiness wash over her."
text=punct_rem(text)
text=stemmer(text)
text=tokenizer.texts_to_sequences([text])
prediction=model.predict(text)
print(prediction)
prediction=np.argmax(prediction)
label_name[prediction]

1/1 [==============================] - 0s 89ms/step
[[0.00107445 0.9229025  0.01295861 0.00973141 0.04515449 0.00817856]]


'joy'

In [ ]:
text="In the tender moments shared between them, each glance exchanged spoke volumes, conveying a depth of affection that transcended words."
text=punct_rem(text)
text=stemmer(text)
text=tokenizer.texts_to_sequences([text])
prediction=model.predict(text)
print(prediction)
prediction=np.argmax(prediction)
label_name[prediction]

1/1 [==============================] - 0s 54ms/step
[[4.0056901e-03 3.7576724e-02 9.5835537e-01 5.9888378e-05 7.2739645e-08
  2.2767804e-06]]


'love'

In [ ]:
text="Every creak of the floorboards sent shivers down her spine, her heart racing with apprehension as she navigated the darkened corridors of the abandoned mansion."
text=punct_rem(text)
text=stemmer(text)
text=tokenizer.texts_to_sequences([text])
prediction=model.predict(text)
print(prediction)
prediction=np.argmax(prediction)
label_name[prediction]

1/1 [==============================] - 0s 35ms/step
[[2.8939742e-01 1.6377514e-03 1.6531064e-06 3.8466111e-03 7.0511431e-01
  2.2402180e-06]]


'fear'

In [ ]:
text="Every rustle of leaves sent a chill down her spine, her senses on high alert as she navigated the eerie forest, fearing the unknown lurking in the shadows."
text=punct_rem(text)
text=stemmer(text)
text=tokenizer.texts_to_sequences([text])
prediction=model.predict(text)
print(prediction)
prediction=np.argmax(prediction)
label_name[prediction]

1/1 [==============================] - 0s 59ms/step
[[3.9188024e-02 1.9542435e-04 2.2912266e-08 2.8259710e-03 9.5778906e-01
  1.4994554e-06]]


'fear'

In [ ]:
text="With bated breath, they watched as the magician waved his wand, transforming a simple hat into a flurry of colorful butterflies, leaving the audience in stunned silence."
text=punct_rem(text)
text=stemmer(text)
text=tokenizer.texts_to_sequences([text])
prediction=model.predict(text)
print(prediction)
prediction=np.argmax(prediction)
label_name[prediction]

1/1 [==============================] - 0s 57ms/step
[[1.7628573e-02 5.2537275e-03 7.9279559e-08 3.5263600e-03 4.3094394e-01
  5.4264730e-01]]


'surprise'

In [ ]:
text="With each word of insult hurled at him, his temper flared hotter, his eyes blazing with righteous indignation at the blatant disrespect."
text=punct_rem(text)
text=stemmer(text)
text=tokenizer.texts_to_sequences([text])
prediction=model.predict(text)
print(prediction)
prediction=np.argmax(prediction)
label_name[prediction]

1/1 [==============================] - 0s 58ms/step
[[1.1657801e-02 1.6029156e-04 2.6362915e-08 9.8814768e-01 3.4234705e-05
  9.0945709e-11]]


'anger'